In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import dataset_prep

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("../data/DoS_0709_new_columns.csv",
        index_col=False,
    )

In [4]:
dataset_prep.clean_dataset(data)
data = dataset_prep.sample_dataset(data, 50000)

Nombre de lignes avant nettoyage :  4753433
Nombre de lignes après nettoyage :  4518383


In [5]:
data["label"].value_counts()

13    28668
0     21332
Name: label, dtype: int64

## Avec les colonnes selectionnées par Random Forest

In [6]:
from sklearn.model_selection import train_test_split

# Colonnes sélectionnées par la méthode Random Forest
selected_columns = ["time_diff",
"nb_packets_sent",
"max_speed1",
"max_speed2",
"hed_noise_x",
"sendTime",
"rcvTime",
"hed_noise_y",
]

X = np.array(data[selected_columns])
y = np.array(data["label"])

for i in range(len(y)):
    if y[i] == 13:
        y[i] = 1

# Séparation en données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [8]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [17]:
from sklearn.metrics import mean_absolute_error

y_pred = my_model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

acc_xgb = (y_pred == y_test).sum().astype(float) / len(y_pred)*100

print("XGBoost's prediction accuracy is: %3.2f" % (acc_xgb))
print("Mean Absolute Error : " + str(mean_absolute_error(y_pred,y_test)))

XGBoost's prediction accuracy is: 99.80
Mean Absolute Error : 0.002


## Naive Bayes

In [18]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Prédiction
y_pred = gnb.predict(X_test)

acc_gnb = (y_pred == y_test).sum().astype(float) / len(y_pred)*100

print("Scikit-Learn's Gaussian Naive Bayes Classifier's prediction accuracy is: %3.2f" % (acc_gnb))

Scikit-Learn's Gaussian Naive Bayes Classifier's prediction accuracy is: 80.82


In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.73      0.86      0.79      2116
           1       0.88      0.77      0.82      2884

    accuracy                           0.81      5000
   macro avg       0.81      0.82      0.81      5000
weighted avg       0.82      0.81      0.81      5000



## Super Vector Machine Classifier

In [21]:
from sklearn.svm import SVC

svc = SVC()

svc.fit(X_train, y_train)
preds = svc.predict(X_test)

acc_svc = (preds == y_test).sum().astype(float) / len(preds)*100

print("Scikit-Learn's Support Vector Machine Classifier's prediction accuracy is: %3.2f" % (acc_svc))

Scikit-Learn's Support Vector Machine Classifier's prediction accuracy is: 82.04
